In [ ]:
# https://gist.github.com/korakot/5c8e21a5af63966d80a676af0ce15067
# install chromium, its driver, and selenium
!apt install chromium-chromedriver
!pip install selenium

In [2]:
# https://gist.github.com/pybites/0aa6d9833849a0942ed218b1d46c47b4#file-platform_login_selenium-py
import os
from IPython.display import HTML
# make venv and pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


USERNAME ="YOUR_USERNAME"
PASSWORD = "YOUR_PASSWORD"
BITE_NUM = 101
BASE_URL = "https://codechalleng.es"
LOGIN_URL = f"{BASE_URL}/login/"
PYBITES_BITE_URL = f"{BASE_URL}/bites/{BITE_NUM}"
WAIT_SECONDS = 2
CODE_TEMPLATE = ""
TEST_NAME = ""
CODE_TEST = ""


# set your codechalleng.es username and password in venv/bin/activate, then source it
#USERNAME = os.getenv('PB_USER')
#PASSWORD = os.getenv('PB_PW')

# set options to be headless, ..
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


# open it, go to a website, and get results
with webdriver.Chrome('chromedriver',options=options) as driver:
  driver.get(LOGIN_URL)
  
  driver.find_element_by_name('username').send_keys(USERNAME)
  driver.find_element_by_name('password').send_keys(PASSWORD + Keys.RETURN)
  
  # I am logged in, let go to a Bite
  driver.get(PYBITES_BITE_URL)
  # do your thing
  #_html = driver.page_source
  #print(_html)
  driver.implicitly_wait(WAIT_SECONDS)
  element = driver.find_element_by_name("template_code")
  CODE_TEMPLATE = element.get_attribute("value")
  element = driver.find_element_by_xpath("//a[@data-mui-controls='pane-default-2']/span")
  element.click()
  TEST_NAME = element.text
  TEST_NAME = TEST_NAME.strip("(").strip(")")
  driver.execute_script("mui.tabs.activate('pane-default-2')")
  # Simular Copy/Paste en Selenium
  # Copy
  cb = driver.execute_script("return copyTestToClipBoard();")
  alert = driver.switch_to.alert
  alert.accept()
  driver.switch_to.default_content
  # Crear textArea
  js = """
  function createTextArea() {
        var tArea = document.createElement('textarea');
        tArea.id = 'myTestCode';
        var pane = document.getElementById('pane-default-2');
        pane.appendChild(tArea);
  }
  createTextArea();
  """
  cb = driver.execute_script(js)
  # Paste
  textAreaElement = driver.find_element_by_id("myTestCode")
  textAreaElement.click()
  textAreaElement.send_keys(Keys.CONTROL + "v")
  CODE_TEST = textAreaElement.get_property("value")


print(TEST_NAME.split("test_")[1].upper() + "\n")
print(CODE_TEMPLATE + "\n\n\n\n")
print(TEST_NAME.upper() + "\n")
print(CODE_TEST)
#print(driver.page_source)




DRIVING.PY

MIN_DRIVING_AGE = 18


def allowed_driving(name, age):
    """Print '{name} is allowed to drive' or '{name} is not allowed to drive'
       checking the passed in age against the MIN_DRIVING_AGE constant"""
    pass




TEST_DRIVING.PY

from driving import allowed_driving


def test_not_allowed_to_drive(capfd):
    allowed_driving('tim', 17)
    output = capfd.readouterr()[0].strip()
    assert output == 'tim is not allowed to drive'


def test_allowed_to_drive(capfd):
    allowed_driving('bob', 18)
    output = capfd.readouterr()[0].strip()
    assert output == 'bob is allowed to drive'


def test_allowed_to_drive_other_name(capfd):
    allowed_driving('julian', 19)
    output = capfd.readouterr()[0].strip()
    assert output == 'julian is allowed to drive'
